### A. Librerias

In [1]:
import os
import time
import json

import pandas as pd

from utils.utils import get_chrome_driver,stop_chrome_driver
from utils.paths import CHROMEDRIVER_PATH, PROJECT_PATH

ESTAMOS EN --->  darwin
PROJECT_PATH:  /Users/nahumfg/Projects/GitHubProjects/NexoInmobiliarioScrapper
CHROME_DRIVER_PATH:  /Users/nahumfg/Projects/GitHubProjects/NexoInmobiliarioScrapper/chromedriver/darwin/chromedriver



### B. Funciones

In [7]:
def get_files(path, extension):
    dir_name  = os.path.join(os.getcwd(), path)
    files     = os.listdir(dir_name)
    
    paths    = []
    for file in files:
        if extension in file:
            paths.append(os.path.join(path,file))

    return paths

def start_har(proxy, name):
    options = {
        'captureHeaders': True, 
        'captureContent': True, 
        'captureBinaryContent': True, 
        'captureCookies': True, 
        'captureHeadersSize': -1, 
        'captureMaxSize': -1, 
        'captureBinaryContentMaxLength': -1 # Tamaño máximo del contenido binario
    }
    proxy.new_har(name, options=options)

def clean_files(path, extension='.json'):
    try:
        files = get_files(path, extension)
        for file in files:
            os.remove(file)
        print('Se eliminaron los archivos de la carpeta')
    except:
        pass

### C. Ejecución - Buscar departamentos por distritos

In [8]:
# 1. Seleccionar modo
HEADLESS = False
PRINT_VIEW = False
TIME_SLEEP = 2

# 2. Eliminar todos los archivos de la carpeta
distritos_urls = [
                  ['surquillo','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-surquillo-lima-lima-150141']
                , ['miraflores','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-lotes-o-casas-en-miraflores-lima-lima-150122']
                , ['barranco','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-barranco-lima-lima-150104']
                , ['breña','https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-bre%C3%B1a-lima-lima-150105']
                , ['jesus_maria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-jesus-maria-lima-lima-150113']
                , ['lince', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-lince-lima-lima-150116']
                , ['magdalena', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-magdalena-del-mar-lima-lima-150120']
                , ['pueblo_libre', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-pueblo-libre-lima-lima-150121']
                , ['san_isidro', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-isidro-lima-lima-150131']
                , ['san_miguel', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
                , ['la_victoria', 'https://nexoinmobiliario.pe/busqueda/venta-de-departamentos-o-oficinas-o-casas-o-lotes-en-san-miguel-lima-lima-150136']
            ]

# clean_files(os.path.join(os.getcwd(),'originals','1_inicio'))

In [ ]:
for distrito, url in distritos_urls:
    # 0. Eliminar todos los archivos de la carpeta
    clean_files(os.path.join(os.getcwd(),'originals','2_distritos',distrito))
    
    # 1. Iniciar el driver, el servidor proxy y el proxy
    driver, server, proxy = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=PRINT_VIEW, headless=HEADLESS)

    # 2. Iniciar har
    start_har(proxy, 'nexo')

    # 3. Abrir la página
    driver.get(url)

    # 4. Guardar html
    time.sleep(TIME_SLEEP)
    html = driver.page_source

    with open(os.path.join(os.getcwd(),'originals','1_inicio', distrito + '.html'), 'w') as outfile:
        outfile.write(html)

    # 5. Cerrar el driver
    stop_chrome_driver(driver, server, proxy)

### D. Ejecución - Listar departamentos

In [12]:
urls_list = []
for distrito, x in distritos_urls:
    # 0. Leer el html
    with open(os.path.join(os.getcwd(),'originals','1_inicio','surquillo.html'), 'r') as infile:
        lines = infile.readlines()

    # 1. Buscar la línea que contiene la palabra 'url'
    cadena = ''
    for i in range(len(lines)-1,0,-1):
        if '"url":' in lines[i]:
            cadena = lines[i]
            break
    
    # 2. Buscar las urls
    posiciones_url = []
    posicion = cadena.find('"url":')

    while posicion != -1:
        posiciones_url.append(posicion)
        posicion = cadena.find('"url":', posicion + 1)
    
    # 3. Buscar los project_id
    posiciones_ids = []
    posicion = cadena.find('"project_id":')

    while posicion != -1:
        posiciones_ids.append(posicion)
        posicion = cadena.find('"project_id":', posicion + 1)

    # 4. Buscar links
    for a,b in zip(posiciones_url, posiciones_ids):
        url = ''.join(list(cadena)[a + 7 :b - 2])
        url = url.replace('\x5C','')  
        url = str(url)

        base = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-' + url.split('/')[-1]
        urls_list.append(base)

len(urls_list)
    

308

### E. Ejecución - Buscar cada departamento

In [2]:
import requests
from bs4 import BeautifulSoup

url = 'https://nexoinmobiliario.pe/proyecto/venta-de-departamento-2725-eres-fase-2'
response = requests.get(url)
print(response.status_code)

200


In [16]:
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra el elemento <p> con la clase 'Project-header-address'
    elemento = soup.find('p', class_='Project-header-address')

    # Verifica si el elemento fue encontrado antes de intentar acceder al texto
    if elemento:
        # Obtiene el texto del elemento
        texto_del_elemento = elemento.get_text(strip=True)

        # Imprime el texto o realiza las operaciones necesarias
        print(texto_del_elemento)
    else:
        print("Elemento no encontrado.")
else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")

Av. República de Panamá 4077


In [17]:
# Verifica que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extrae el texto de <h1>Eres Fase 2</h1>
    titulo = soup.find('h1').get_text(strip=True)
    print(f'Título: {titulo}')

    # Extrae el texto de <p class="Project-header-address">Av. República de Panamá 4077</p>
    direccion = soup.find('p', class_='Project-header-address').get_text(strip=True)
    print(f'Dirección: {direccion}')

    # Extrae el texto de <p class="Project-header-address"> - Surquillo</p>
    distrito = soup.find('p', class_='Project-header-address').get_text(strip=True).replace('-', '').strip()
    print(f'Distrito: {distrito}')

else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")


Título: Eres Fase 2
Dirección: Av. República de Panamá 4077
Distrito: Av. República de Panamá 4077


In [18]:
# Verifica que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Extrae el texto de <h1>Eres Fase 2</h1>
    titulo = soup.find('h1').get_text(strip=True)
    print(f'Título: {titulo}')

    # Extrae el texto de <p class="Project-header-address">Av. República de Panamá 4077</p>
    direccion = soup.find('p', class_='Project-header-address').get_text(strip=True)
    print(f'Dirección: {direccion}')

    # Extrae el texto de <p class="Project-header-address"> - Surquillo</p>
    distrito = soup.find('p', class_='Project-header-address').find_next_sibling('p').get_text(strip=True).replace('-', '').strip()
    print(f'Distrito: {distrito}')

    # Extrae el precio desde <p class="Project-header-price"><span class="title-price">Precio desde</span> <strong> S/. 319,118</strong></p>
    precio_desde = soup.find('p', class_='Project-header-price').strong.get_text(strip=True)
    print(f'Precio desde: {precio_desde}')

    # Extrae el valor de referencia <p class="title-price">(<span>valor referencia </span><strong>$ 84,940</strong>)</p>
    valor_referencia = soup.find('p', class_='title-price').strong.get_text(strip=True)
    print(f'Valor de referencia: {valor_referencia}')

else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")


Título: Eres Fase 2
Dirección: Av. República de Panamá 4077
Distrito: Surquillo
Precio desde: S/. 319,118
Valor de referencia: $ 84,940


In [6]:
# Verifica que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra el div con la clase "bx-data-project box-st"
    div_informacion_proyecto = soup.find('div', class_='bx-data-project box-st')

    if div_informacion_proyecto:
        # Encuentra todas las filas de la tabla dentro del div
        filas_tabla = div_informacion_proyecto.find('table').find_all('tr')

        # Itera sobre las filas de la tabla e imprime la información
        for fila in filas_tabla:
            columnas = fila.find_all('td')
            if len(columnas) == 2:
                try:
                    etiqueta = columnas[0].strong.get_text(strip=True)
                    valor = columnas[1].get_text(strip=True)
                    print(f'{etiqueta}: {valor}')
                except:
                    pass

        # También puedes extraer información específica usando el método find para elementos específicos
        # Por ejemplo, para obtener la información de la inmobiliaria
        inmobiliaria_logo = div_informacion_proyecto.find('tr', class_='Project-inmobiliaria').find('img')['src']
        inmobiliaria_nombre = div_informacion_proyecto.find('tr', class_='Project-inmobiliaria').find('h2').get_text(strip=True)

        print(f'Inmobiliaria Logo: {inmobiliaria_logo}')
        print(f'Inmobiliaria Nombre: {inmobiliaria_nombre}')

    else:
        print("Div no encontrado.")

else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")

Tipo de inmueble: Departamento
Área total: 41.18 a 80.59 m2
Dormitorios: 1 a 3
Etapa del proyecto: En planos
Fecha de entrega: 30 de Septiembre, 2025
Financiamiento: Banco BCP
Inmobiliaria Logo: https://e.nexoinmobiliario.pe/customers/grupo-lar/logo-225-20231027061803.jpg
Inmobiliaria Nombre: GRUPO LAR


In [7]:
# Verifica que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra el div con la clase "Project-menu"
    div_menu = soup.find('div', class_='Project-menu')

    if div_menu:
        # Encuentra todos los botones dentro del div
        botones = div_menu.find_all('button')

        # Itera sobre los botones e imprime la información
        for boton in botones:
            valor = boton['value']
            clase = boton.get('class', [])
            estado = 'activo' if 'active' in clase else 'inactivo'
            icono = boton.find('i')['class'][1] if boton.find('i') else ''

            print(f'Valor: {valor}, Estado: {estado}, Icono: {icono}')

    else:
        print("Div no encontrado.")

else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")

Valor: fotos, Estado: activo, Icono: fa-camera
Valor: mapa, Estado: inactivo, Icono: fa-map-marker-alt
Valor: streetview, Estado: inactivo, Icono: fa-street-view


In [10]:
# Verifica que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra el div con la clase "Project-available-model"
    div_modelo_disponible = soup.find('div', class_='Project-available-model')

    if div_modelo_disponible:
        # Extrae información del enlace de imagen
        imagen_enlace = div_modelo_disponible.find('a', class_='popup-link')['data-links'].strip()
        print(f'Enlace de la imagen: {imagen_enlace}')

        # Extrae información del modelo
        nombre_modelo = div_modelo_disponible.find('span', class_='name_tipology').get_text(strip=True).strip()
        print(f'Nombre del modelo: {nombre_modelo}')

        # Extrae información del número de pisos
        num_pisos = div_modelo_disponible.find('span', class_='num_pisos').get_text(strip=True).strip()
        print(f'Número de pisos: {num_pisos}')

        # Extrae información del dormitorio, área y precio
        dormitorio = div_modelo_disponible.find('span', class_='bedroom').get_text(strip=True).strip()
        area = div_modelo_disponible.find('span', class_='area').get_text(strip=True).strip()
        precio = div_modelo_disponible.find('span', class_='price').get_text(strip=True).strip()

        print(f'Dormitorio: {dormitorio}')
        print(f'Área: {area}')
        print(f'Precio desde: {precio}')

        # También puedes extraer información del enlace de cotización
        enlace_cotizacion = div_modelo_disponible.find('a', class_='Btn-ver--red')['href'].strip()
        print(f'Enlace de cotización: {enlace_cotizacion}')

    else:
        print("Div no encontrado.")

else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")

Enlace de la imagen: grupo-lar/2725-eres-fase-2/planos-eres-fase-2-63e10d808961d_b.jpg
Nombre del modelo: A1
Número de pisos: Pisos: Entre 2 al  9
Dormitorio: 1
Área: 52.55                                                m2
Precio desde: S/ 442,532
Enlace de cotización: https://nexoinmobiliario.pe/cotizar/2725-eres-fase-2-55110


In [11]:
# Verifica que la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra todos los divs con la clase "Project-available-model"
    modelos_disponibles = soup.find_all('div', class_='Project-available-model')

    for modelo in modelos_disponibles:
        # Extrae información del enlace de imagen
        imagen_enlace = modelo.find('a', class_='popup-link')['data-links'].strip()
        print(f'Enlace de la imagen: {imagen_enlace}')

        # Extrae información del modelo
        nombre_modelo = modelo.find('span', class_='name_tipology').get_text(strip=True).strip()
        print(f'Nombre del modelo: {nombre_modelo}')

        # Extrae información del número de pisos
        num_pisos = modelo.find('span', class_='num_pisos').get_text(strip=True).strip()
        print(f'Número de pisos: {num_pisos}')

        # Extrae información del dormitorio, área y precio
        dormitorio = modelo.find('span', class_='bedroom').get_text(strip=True).strip()
        area = modelo.find('span', class_='area').get_text(strip=True).strip()
        precio = modelo.find('span', class_='price').get_text(strip=True).strip()

        print(f'Dormitorio: {dormitorio}')
        print(f'Área: {area}')
        print(f'Precio desde: {precio}')

        # También puedes extraer información del enlace de cotización
        enlace_cotizacion = modelo.find('a', class_='Btn-ver--red')['href'].strip()
        print(f'Enlace de cotización: {enlace_cotizacion}')

        print("\n---\n")  # Separador entre modelos

else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")

Enlace de la imagen: grupo-lar/2725-eres-fase-2/planos-eres-fase-2-63e10d808961d_b.jpg
Nombre del modelo: A1
Número de pisos: Pisos: Entre 2 al  9
Dormitorio: 1
Área: 52.55                                                m2
Precio desde: S/ 442,532
Enlace de cotización: https://nexoinmobiliario.pe/cotizar/2725-eres-fase-2-55110

---

Enlace de la imagen: grupo-lar/2725-eres-fase-2/planos-eres-fase-2-63e10d9c1eafa_b.jpg
Nombre del modelo: A2
Número de pisos: Pisos: Entre 2 al  11
Dormitorio: 2
Área: 67.64                                                m2
Precio desde: S/ 532,915
Enlace de cotización: https://nexoinmobiliario.pe/cotizar/2725-eres-fase-2-55111

---

Enlace de la imagen: grupo-lar/2725-eres-fase-2/planos-eres-fase-2-63e10dc231fa1_b.jpg
Nombre del modelo: A3
Número de pisos: Pisos: Entre 2 al  8
Dormitorio: 1
Área: 50.67                                                m2
Precio desde: S/ 430,756
Enlace de cotización: https://nexoinmobiliario.pe/cotizar/2725-eres-fase-2-55112


In [12]:
if response.status_code == 200:
    # Parsea el contenido HTML de la página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encuentra el campo de entrada para la latitud
    input_latitud = soup.find('input', id='latitude')

    if input_latitud:
        latitud = input_latitud['value'].strip()
        print(f'Latitud: {latitud}')

    # Encuentra el campo de entrada para la longitud
    input_longitud = soup.find('input', id='longitude')

    if input_longitud:
        longitud = input_longitud['value'].strip()
        print(f'Longitud: {longitud}')

else:
    print(f"Error al obtener la página. Código de estado: {response.status_code}")

Latitud: -12.1040361
Longitud: -77.0184555


In [13]:
def generar_url_google_maps(latitud, longitud):
    url = f"https://www.google.com/maps?q={latitud},{longitud}"
    return url

In [14]:
def generar_url_street_view(latitud, longitud):
    url = f"https://www.google.com/maps?q=&layer=c&cbll={latitud},{longitud}"
    return url

# Ejemplo de uso
latitud = -12.1040361
longitud = -77.0184555

url_street_view = generar_url_street_view(latitud, longitud)
print("URL de Street View:", url_street_view)


URL de Street View: https://www.google.com/maps?q=&layer=c&cbll=-12.1040361,-77.0184555


In [15]:
# 7. Eliminar los archivos que no tienen datos
try:
    os.remove('bmp.log')
    os.remove('server.log')
    print("Se eliminaron los archivos bmp.log y server.log")
except:
    pass

Se eliminaron los archivos bmp.log y server.log
